# Bernoulli Console With Random Orientation and Local Distributed Loads

In [5]:
L, d, t = 1000.0, 100.0, 5.0
Ex, nu = 210000.0, 0.25
penalty = 1e40  # penalty value


## Linear Solution

In [6]:
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector, linspace
from neumann.array import repeat
from polymesh.space import StandardFrame, PointCloud
from polymesh.space.utils import index_of_closest_point, index_of_furthest_point
from polymesh.topo.tr import L2_to_L3
from sigmaepsilon.solid.fem.cells import B2, B3
from sigmaepsilon.solid import Structure, LineMesh, PointData, BeamSection
import numpy as np
from latexdocs.utils import floatformatter

seed = 0  # integer or None
seed = None

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)

formatter = floatformatter(sig=6)
def f2s(x): return formatter.format(x)


# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)


# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])


def solve(n, angles, loads, celltype=B2):
    # space
    GlobalFrame = StandardFrame(dim=3)
    TargetFrame = GlobalFrame.rotate('Body', angles, 'XYZ', inplace=False)

    # mesh
    p0 = np.array([0., 0., 0.])
    p1 = np.array([L, 0., 0.])
    coords = linspace(p0, p1, n+1)
    points = PointCloud(coords, frame=TargetFrame)
    coords = points.show(GlobalFrame)
    topo = np.zeros((n, 2), dtype=int)
    topo[:, 0] = np.arange(n)
    topo[:, 1] = np.arange(n) + 1
    if celltype.NNODE == 3:
        coords, topo = L2_to_L3(coords, topo)
    i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
    i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

    # essential boundary conditions
    fixity = np.zeros((coords.shape[0], 6)).astype(bool)
    fixity[i_first, :] = True
    fixity = fixity.astype(float) * penalty

    # natural boundary conditions
    loads = np.array(loads)
    nodal_loads = np.zeros((coords.shape[0], 6))
    cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6))
    cell_loads[:, :, :] = loads

    # pointdata
    pd = PointData(coords=coords, frame=GlobalFrame,
                   loads=nodal_loads, fixity=fixity)

    # celldata
    frames = repeat(TargetFrame.axes, topo.shape[0])
    cd = celltype(topo=topo, loads=cell_loads, frames=frames)

    # set up mesh and structure
    mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
    structure = Structure(mesh=mesh)

    structure.linsolve()

    dofsol = structure.nodal_dof_solution(store='dofsol')
    u = np.zeros(6)
    u[:3] = Vector(dofsol[i_last, :3], frame=GlobalFrame).show(TargetFrame)
    u[3:] = Vector(dofsol[i_last, 3:], frame=GlobalFrame).show(TargetFrame)

    reactions = structure.reaction_forces()
    r = np.zeros(6)
    r[:3] = Vector(reactions[i_first, :3], frame=GlobalFrame).show(TargetFrame)
    r[3:] = Vector(reactions[i_first, 3:], frame=GlobalFrame).show(TargetFrame)

    forces = structure.internal_forces()
    f = np.zeros(6)
    f[:3] = forces[0, 0, :3]
    f[3:] = forces[0, 0, 3:]

    return u, r, f


## Tests

In [7]:
def absolute_relative_error(v_analytic, v_fem):
    return np.abs(100 * (v_analytic - v_fem) / v_analytic)


### Constant distributed forces

In [8]:
qx, qy, qz, qxx = 1.0, 1.0, 1.0, 1.0
loads = [qx, qy, qz, qxx, 0, 0]
n = 2  # number of elements
cells = [B2, B3]

# analytical solutions
UX = qx * L**2 / (2 * Ex * A)  # displacement at the free end
UY = qy * L**4 / (8 * Ex * Iz)  # displacement at the free end
UZ = qz * L**4 / (8 * Ex * Iy)  # displacement at the free end
UXX = qxx * L**2 / (2 * G * Ix)  # rotation at the free end
UYY = - qz * L**3 / (6 * Ex * Iy)  # rotation at the free end
UZZ = qy * L**3 / (6 * Ex * Iz)  # rotation at the free end
RX = - qx * L  # reaction force
RY = - qy * L  # reaction force
RZ = - qz * L  # reaction force
RXX = - qxx * L  # reaction moment
RYY = 0.5 * qy * L ** 2  # reaction moment
RZZ = - 0.5 * qy * L ** 2  # reaction moment
FX = qx * L  # internal force at the support
FY = qy * L  # internal force at the support
FZ = qz * L  # internal force at the support
FXX = qxx * L  # internal moment at the support
FYY = -0.5 * qy * L ** 2  # internal moment at the support
FZZ = 0.5 * qy * L ** 2 * L  # internal moment at the support

for cell in cells:
    for i in range(3):
        angles = np.random.rand(3) * np.pi * 2
        u, r, f = solve(n, angles, loads, cell)
        UX_fem, UXX_fem = u[0], u[3]
        RX_fem, RXX_fem = r[0], r[3]
        FX_fem, FXX_fem = f[0], f[3]
        UY_fem, UZZ_fem = u[1], u[5]
        RY_fem, RZZ_fem = r[1], r[5]
        FY_fem, FZZ_fem = f[1], f[5]
        UZ_fem, UYY_fem = u[2], u[4]
        RZ_fem, RYY_fem = r[2], r[4]
        FZ_fem, FYY_fem = f[2], f[4]
        print("---------------------- B{} ------------------\n".format(cell.NNODE))
        print("angles : {}".format(angles))
        print("UX  | Analytic : {}, FEM : {}".format(f2s(UX), f2s(UX_fem)))
        print("UY  | Analytic : {}, FEM : {}".format(f2s(UY), f2s(UY_fem)))
        print("UZ  | Analytic : {}, FEM : {}".format(f2s(UZ), f2s(UZ_fem)))
        print("UXX | Analytic : {}, FEM : {}".format(f2s(UXX), f2s(UXX_fem)))
        print("UYY | Analytic : {}, FEM : {}".format(f2s(UYY), f2s(UYY_fem)))
        print("UZZ | Analytic : {}, FEM : {}".format(f2s(UZZ), f2s(UZZ_fem)))
        print("RX  | Analytic : {}, FEM : {}".format(f2s(RX), f2s(RX_fem)))
        print("RY  | Analytic : {}, FEM : {}".format(f2s(RY), f2s(RY_fem)))
        print("RZ  | Analytic : {}, FEM : {}".format(f2s(RZ), f2s(RZ_fem)))
        print("RXX | Analytic : {}, FEM : {}".format(f2s(RXX), f2s(RXX_fem)))
        print("RYY | Analytic : {}, FEM : {}".format(f2s(RYY), f2s(RYY_fem)))
        print("RZZ | Analytic : {}, FEM : {}".format(f2s(RZZ), f2s(RZZ_fem)))
        print("FX  | Analytic : {}, FEM : {}".format(f2s(FX), f2s(FX_fem)))
        print("FY  | Analytic : {}, FEM : {}".format(f2s(FY), f2s(FY_fem)))
        print("FZ  | Analytic : {}, FEM : {}".format(f2s(FZ), f2s(FZ_fem)))
        print("FXX | Analytic : {}, FEM : {}".format(f2s(FXX), f2s(FXX_fem)))
        print("FYY | Analytic : {}, FEM : {}".format(f2s(FYY), f2s(FYY_fem)))
        print("FZZ | Analytic : {}, FEM : {}".format(f2s(FZZ), f2s(FZZ_fem)))
        print("\n")


---------------------- B2 ------------------

angles : [5.98479897 3.31209111 3.64544628]
UX  | Analytic : 0.00160584, FEM : 0.00160584
UY  | Analytic : 0.357169, FEM : 0.347247
UZ  | Analytic : 0.357169, FEM : 0.347247
UXX | Analytic : 1.78584e-06, FEM : 1.78584e-06
UYY | Analytic : -0.000476225, FEM : -0.000456382
UZZ | Analytic : 0.000476225, FEM : 0.000456382
RX  | Analytic : -1000, FEM : -1000
RY  | Analytic : -1000, FEM : -1000
RZ  | Analytic : -1000, FEM : -1000
RXX | Analytic : -1000, FEM : -1000
RYY | Analytic : 500000, FEM : 500000
RZZ | Analytic : -500000, FEM : -500000
FX  | Analytic : 1000, FEM : 750
FY  | Analytic : 1000, FEM : 750
FZ  | Analytic : 1000, FEM : 750
FXX | Analytic : 1000, FEM : 750
FYY | Analytic : -500000, FEM : -472222
FZZ | Analytic : 5e+08, FEM : 472222


---------------------- B2 ------------------

angles : [2.31735585 3.11583478 1.89825998]
UX  | Analytic : 0.00160584, FEM : 0.00160584
UY  | Analytic : 0.357169, FEM : 0.347247
UZ  | Analytic : 0.3571